In [2]:
import geopandas as gpd

In [4]:
df = gpd.read_parquet('../data/all_sites_20240703.parquet')

In [5]:
df.head()

region     name    area_ha   epsg  \
0    usa  usda_me  39.509840  26919   
1    usa  usda_me  74.565161  26919   
2    usa  usda_me  33.698329  26919   
3    usa  usda_me  59.362166  26919   
4    usa  usda_me  25.889235  26919   

                                            geometry  \
0  POLYGON ((-69.91152 46.61153, -69.90748 46.611...   
1  POLYGON ((-69.2508 46.0587, -69.24307 46.0587,...   
2  POLYGON ((-69.38241 46.02053, -69.37891 46.020...   
3  POLYGON ((-69.31656 45.6049, -69.31036 45.6049...   
4  POLYGON ((-70.22245 45.79862, -70.21976 45.798...   

                            file  
0   usa_usda_me_me_47_43.parquet  
1  usa_usda_me_me_91_146.parquet  
2  usa_usda_me_me_84_153.parquet  
3  usa_usda_me_me_87_230.parquet  
4  usa_usda_me_me_29_193.parquet

In [6]:
dissolved = df.dissolve('name')

In [7]:
dissolved

geometry  \
name                                                                            
amani                       MULTIPOLYGON (((38.57661 -5.1684, 38.57661 -5....   
ara_smapex                  MULTIPOLYGON (((146.01908 -34.81508, 146.01944...   
aso_sierra_nevada           POLYGON ((-119.49575 37.07304, -119.49575 37.0...   
aso_sierra_nevada_snowoff   POLYGON ((-119.5135 37.11449, -119.49664 37.11...   
boschetti_idaho_clearcreek  MULTIPOLYGON (((-115.80007 45.94813, -115.7871...   
...                                                                       ...   
whroo                       POLYGON ((145.02367 -36.67753, 145.02367 -36.6...   
zhangye_china               POLYGON ((100.24594 38.53489, 100.24525 38.534...   
zigzag_creek                MULTIPOLYGON (((148.31063 -37.49349, 148.31063...   
zofin_180416                MULTIPOLYGON (((14.69868 48.66433, 14.69841 48...   
zofin_180607                MULTIPOLYGON (((14.69868 48.663, 14.69868 48.6...   

                               region     area_ha   epsg  \
name                                                       
amani                          africa   56.188534  32736   
ara_smapex                  australia  240.216035  28355   
aso_sierra_nevada                 usa  397.649228  32611   
aso_sierra_nevada_snowoff         usa  399.047587  32611   
boschetti_idaho_clearcreek        usa  206.987956  26911   
...                               ...         ...    ...   
whroo                       australia  154.950304  28355   
zhangye_china                    asia    0.582617  32647   
zigzag_creek                australia  106.335828  28355   
zofin_180416                   europe    4.958711   5514   
zofin_180607                   europe    4.957908   5514   

                                                                         file  
name                                                                           
amani                                               africa_amani_0083.parquet  
ara_smapex                  australia_ara_smapex_0905LS21crYB1_2a_ahd.parquet  
aso_sierra_nevada           usa_aso_sierra_nevada_Sierra_SanJoaquim_KingsC...  
aso_sierra_nevada_snowoff   usa_aso_sierra_nevada_snowoff_Sierra_SanJoaqui...  
boschetti_idaho_clearcreek    usa_boschetti_idaho_clearcreek_pt000030.parquet  
...                                                                       ...  
whroo                       australia_whroo_tile_323404_5939080_all_000000...  
zhangye_china                           asia_zhangye_china_yd03_shift.parquet  
zigzag_creek                australia_zigzag_creek_tile_618859_5849222_all...  
zofin_180416                europe_zofin_180416_Zofin_04162018_-743700_-12...  
zofin_180607                europe_zofin_180607_Zofin_06072018_-743400_-12...  

[453 rows x 5 columns]

In [18]:
import pyproj
import geopandas as gpd
# Define a function to calculate area per row
def calculate_area(row):
    # Determine UTM zone based on longitude
    utm_zone = (int((row.geometry.centroid.x + 180) / 6) % 60) + 1
    epsg_code = 32600 + utm_zone  # UTM for Northern Hemisphere

    # Create a single-row GeoDataFrame
    projected_gdf = gpd.GeoDataFrame([row], columns=dissolved.columns, crs=dissolved.crs)

    # Reproject to the UTM zone
    projected_gdf = projected_gdf.to_crs(epsg=epsg_code)

    # Compute area in hectares
    return projected_gdf.geometry.area.values[0] / 10_000  


# Apply the function to each row
dissolved["area_ha"] = dissolved.apply(calculate_area, axis=1)

In [17]:
dissolved[:2].apply(calculate_area, axis=1)

name
amani         13308.396384
ara_smapex     6898.046262
dtype: float64

In [21]:
all_sites_dissolved = dissolved.reset_index().drop(columns=["file"])

In [23]:
all_sites_dissolved.to_file("../data/all_sites_dissolved.gpkg", driver="GPKG")